In [509]:
%load_ext autoreload
%autoreload 2
from ctypes import util
import sys
import copy
from decimal import Decimal

from numpy import sort
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.utils.misc import greedy_allocation_algorithm
from sturdy.utils.lazy import pick_one_allocation_algorithm, sorted_greedy_allocation_algorithm, equal_greedy_allocation_algorithm
from validator.simulator import Simulator
from sturdy.utils.misc import supply_rate

def calculate_apy(simulator: Simulator, assets_and_pools, allocations):
        # reset simulator for next run
    simulator.reset()

    if allocations is None:
        return sys.float_info.min

    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)

    # update reserves given allocations
    # simulator.allocations = allocations
    simulator.update_reserves_with_allocs()
    # TODO: use this or just run reset()?
    updated_assets_pools = copy.deepcopy(simulator.assets_and_pools)

    initial_balance = updated_assets_pools["total_assets"]
    total_allocated = Decimal(0)
    cheating = False

    for _, allocation in allocations.items():
        total_allocated += Decimal(
            str(allocation)
        )  # This should fix precision issues with python floats

        # score response very low if miner is cheating somehow
        if total_allocated > initial_balance:
            cheating = True
            break

    # punish if miner they're cheating
    # TODO: create a more forgiving penalty system?
    if cheating:
        import pdb; pdb.set_trace()
        print(
            f"CHEATER DETECTED - PUNISHING 👊😠"
        )
        return sys.float_info.min

    # run simulation
    simulator.run()
    # calculate aggregate yield
    pct_yield = 0
    for pools in simulator.pool_history:
        curr_yield = 0
        for uid, pool_data in pools.items():
            # print(f'uid: {uid}, pool_data: {pool_data}')
            util_rate = pool_data["borrow_amount"] / pool_data["reserve_size"]
            # util_rate = allocations[uid] / (pool_data['reserve_size'] + allocations[uid])
            pool_yield = allocations[uid] * supply_rate(
                util_rate, simulator.assets_and_pools["pools"][uid]
            )
            # print(f'uid {uid}, reserve: {pool_data["reserve_size"]} allocation: {allocations[uid]} util_rate: {util_rate} yield {pool_yield}')
            curr_yield += pool_yield
        pct_yield += curr_yield
        # print(f'curr_yield: {curr_yield} pct_yield: {pct_yield}')

    pct_yield /= initial_balance
    # print(f'pct_yield after balance: {pct_yield}, steps: {simulator.timesteps} apy: {pct_yield / simulator.timesteps * 365}')
    return(
        pct_yield / simulator.timesteps
    ) * 365  # for simplicity each timestep is a day in the simulator


def calc_strategy_apy(allocations, assets_and_pools):
    simulator = Simulator()
    simulator.initialize()
    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)

    return calculate_apy(simulator, assets_and_pools, allocations)


def strategies_apy(strategies):
    init_assets_and_pools = generate_assets_and_pools()

    result_apy = {}
    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    for k, v in strategies.items():
        allocation = strategies[k](synapse=synapse)
        result_apy[k] = calc_strategy_apy(allocation, init_assets_and_pools)
    return result_apy


def max_strategy():
    init_assets_and_pools = generate_assets_and_pools()

    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    equal_greedy = equal_greedy_allocation_algorithm(synapse=synapse)
    greedy_allocations = greedy_allocation_algorithm(synapse=synapse)
    sorted_greedy = sorted_greedy_allocation_algorithm(synapse=synapse)
    pick_one = pick_one_allocation_algorithm(synapse=synapse)
    print('equal')
    equal_greedy_apy = calc_strategy_apy(equal_greedy, init_assets_and_pools)
    print('original')
    original_apy = calc_strategy_apy(greedy_allocations, init_assets_and_pools)
    print('sorted')
    sorted_greedy_apy = calc_strategy_apy(sorted_greedy, init_assets_and_pools)
    print('pick_one')
    pick_one_apy = calc_strategy_apy(pick_one, init_assets_and_pools)

    max_apy = max(equal_greedy_apy, sorted_greedy_apy, original_apy, pick_one_apy)
    if max_apy != pick_one_apy:
        return greedy_allocations, equal_greedy, sorted_greedy, pick_one, synapse.assets_and_pools
    else:
        return None


def compare_strategy():
    init_assets_and_pools = generate_assets_and_pools()

    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    greedy_allocations = greedy_allocation_algorithm(synapse=synapse)
    sorted_greedy = sorted_greedy_allocation_algorithm(synapse=synapse)
    equal_greedy = equal_greedy_allocation_algorithm(synapse=synapse)
    pick_one = pick_one_allocation_algorithm(synapse=synapse)

    simulator = Simulator()
    simulator.initialize()
    simulator.init_data(copy.deepcopy(init_assets_and_pools), greedy_allocations)

    equal_greedy_apy = calculate_apy(simulator, equal_greedy)
    sorted_greedy_apy = calculate_apy(simulator, sorted_greedy)
    original_apy = calculate_apy(simulator, greedy_allocations)
    pick_one_apy = calculate_apy(simulator, pick_one)

    # print(f'equal: {calculate_apy(simulator, equal_greedy)}')
    # print(f'sorted: {calculate_apy(simulator, sorted_greedy)}')
    # print(f'original: {calculate_apy(simulator, greedy_allocations)}')

    max_apy = max(equal_greedy_apy, sorted_greedy_apy, original_apy, pick_one_apy)
    # if max_apy == equal_greedy_apy:
    #     return 'equal'
    # if max_apy == sorted_greedy_apy:
    #     return 'sorted'
    # if max_apy == original_apy:
    #     return 'original'
    # if max_apy == pick_one_apy:
    #     return 'pick_one'
    if max_apy != pick_one_apy:
        return greedy_allocations, equal_greedy, sorted_greedy, pick_one, synapse.assets_and_pools
    else:
        return None


# print(results.count('equal'))
# print(results.count('sorted'))
# print(results.count('original'))
# print(results.count('pick_one'))

# print(greedy_allocations)
# print(init_assets_and_pools['pools'])

# pool = init_assets_and_pools['pools']['0']
# supply_rate(pool['borrow_amount'] * 1.1 / (0 + pool['reserve_size']), pool)

# equal, sorted_greedy, greedy, pick_one = compare_strategy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [521]:
"""
Compute apy from different strategies and compare the results
"""
import pandas as pd
from sturdy.utils.misc import greedy_allocation_algorithm
from sturdy.utils.lazy import equal_greedy_allocation_algorithm, sorted_greedy_allocation_algorithm
from sturdy.utils.yiop import yiop_allocation_algorithm

strategies = {
    'equal': equal_greedy_allocation_algorithm,
    'greedy': greedy_allocation_algorithm,
    'sorted': sorted_greedy_allocation_algorithm,
    'yiop': yiop_allocation_algorithm
}

results = [strategies_apy(strategies) for _ in range(2000)]
df = pd.DataFrame(results)
df['best'] = df.idxmax(axis=1)

In [522]:
df['best'].value_counts()

best
yiop      672
sorted    567
equal     453
greedy    308
Name: count, dtype: int64

In [492]:
greedy_allocations, equal_greedy, sorted_greedy, pick_one, assets_and_pools = [item for item in results if item != None][0]
simulator = Simulator()
simulator.initialize()
print(f'assets and pools: {assets_and_pools}')
print(pick_one)
print('==equal', equal_greedy)
equal_greedy_apy = calculate_apy(simulator, assets_and_pools, equal_greedy)
print('==sorted')
sorted_greedy_apy = calculate_apy(simulator, assets_and_pools, sorted_greedy)
print('==original')
original_apy = calculate_apy(simulator, assets_and_pools, greedy_allocations)
print('==pick one')
pick_one_apy = calculate_apy(simulator, assets_and_pools, pick_one)

print('==custom')
arr = [2.47262559e-01, 5.11797341e-02, 1.02159377e-01, 1.11038600e-01,
 7.08541043e-02, 0.00000000e+00, 7.54874975e-02, 5.14403109e-18,
 5.88625333e-02, 2.83155594e-01]
allocations = {str(i): arr[i] for i in range(len(arr))}
custom_apy = calculate_apy(simulator, assets_and_pools, allocations)
# custom_apy = calculate_apy(simulator, assets_and_pools, {'0': 0.67, '1': 0.33})

print(assets_and_pools['pools'])
print(f'==> equal: {equal_greedy_apy}, sorted: {sorted_greedy_apy}, original: {original_apy}, pick_one: {pick_one_apy}, custom: {custom_apy}')
print(greedy_allocations)
print(pick_one)

assets and pools: {'total_assets': 1.0, 'pools': {'0': {'pool_id': '0', 'base_rate': 0.03, 'base_slope': 0.087, 'kink_slope': 0.687, 'optimal_util_rate': 0.65, 'borrow_amount': 0.8, 'reserve_size': 1.0}, '1': {'pool_id': '1', 'base_rate': 0.04, 'base_slope': 0.059, 'kink_slope': 0.595, 'optimal_util_rate': 0.65, 'borrow_amount': 0.6, 'reserve_size': 1.0}, '2': {'pool_id': '2', 'base_rate': 0.01, 'base_slope': 0.073, 'kink_slope': 0.53, 'optimal_util_rate': 0.7, 'borrow_amount': 0.85, 'reserve_size': 1.0}, '3': {'pool_id': '3', 'base_rate': 0.01, 'base_slope': 0.011, 'kink_slope': 0.699, 'optimal_util_rate': 0.7, 'borrow_amount': 0.9, 'reserve_size': 1.0}, '4': {'pool_id': '4', 'base_rate': 0.05, 'base_slope': 0.028, 'kink_slope': 0.404, 'optimal_util_rate': 0.7, 'borrow_amount': 0.8, 'reserve_size': 1.0}, '5': {'pool_id': '5', 'base_rate': 0.03, 'base_slope': 0.026, 'kink_slope': 0.539, 'optimal_util_rate': 0.65, 'borrow_amount': 0.6, 'reserve_size': 1.0}, '6': {'pool_id': '6', 'base_r

In [270]:
"""
Examine greedy allocation algorithm
"""

from sturdy.constants import *
def format_num_prec(
    num: float, sig: int = GREEDY_SIG_FIGS, max_prec: int = GREEDY_SIG_FIGS
) -> float:
    return float(f"{{0:.{max_prec}f}}".format(float(format(num, f".{sig}f"))))

assets_and_pools = generate_assets_and_pools()
max_balance = assets_and_pools["total_assets"]
balance = max_balance
pools = assets_and_pools["pools"]

# how much of our assets we have allocated
current_allocations = {k: 0.0 for k, _ in pools.items()}

assert balance >= 0

# run greedy algorithm to allocate assets to pools
while balance > 0:
    # TODO: use np.float32 instead of format()??
    current_supply_rates = {
        k: format_num_prec(
            supply_rate(
                util_rate=v["borrow_amount"]
                / (current_allocations[k] + pools[k]["reserve_size"]),
                pool=v,
            )
        )
        for k, v in pools.items()
    }
    print(f'current_supply_rates: {sum(current_supply_rates.values())}')

    default_chunk_size = format_num_prec(CHUNK_RATIO * max_balance)
    to_allocate = 0

    if balance < default_chunk_size:
        to_allocate = balance
    else:
        to_allocate = default_chunk_size

    balance = format_num_prec(balance - to_allocate)
    assert balance >= 0
    max_apy = max(current_supply_rates.values())
    min_apy = min(current_supply_rates.values())
    apy_range = format_num_prec(max_apy - min_apy)

    print(f'min_apy: {min_apy}, max_apy: {max_apy}, apy_range: {apy_range}')
    alloc_it = current_allocations.items()
    pool_ratio = {}
    for pool_id, _ in alloc_it:
        pool_ratio[pool_id] = format_num_prec(to_allocate * (current_supply_rates[pool_id] - min_apy) / (apy_range))
    print(f'pool ratio: {sum(pool_ratio.values())}')
    print(f'to_allocate init: {to_allocate}')
    pool_delta = {}
    for pool_id, _ in alloc_it:
        delta = format_num_prec(
            to_allocate * ((current_supply_rates[pool_id] - min_apy) / (apy_range)),
        )
        pool_delta[pool_id] = delta
        current_allocations[pool_id] = format_num_prec(
            current_allocations[pool_id] + delta
        )
        to_allocate = format_num_prec(to_allocate - delta)
        print(f'to_allocate: {to_allocate}, delta: {delta}')
    print(f'pool_delta: {sum(pool_delta.values())} {pool_delta}\n')
    assert to_allocate == 0  # should allocate everything from current chunk

# print(current_allocations)
# print(sum(current_allocations.values()))

current_supply_rates: 1.11251429
min_apy: 0.01854118, max_apy: 0.4518, apy_range: 0.43325882
pool ratio: 0.021398350000000003
to_allocate init: 0.01
to_allocate: 0.01, delta: 0.0
to_allocate: 0.009372, delta: 0.000628
to_allocate: 0.00886455, delta: 0.00050745
to_allocate: 0.00874127, delta: 0.00012328
to_allocate: 0.00303038, delta: 0.00571089
to_allocate: 0.00293382, delta: 9.656e-05
to_allocate: 0.00287146, delta: 6.236e-05
to_allocate: 0.00228453, delta: 0.00058693
to_allocate: 0.00206034, delta: 0.00022419
to_allocate: 0.0, delta: 0.00206034
pool_delta: 0.009999999999999998 {'0': 0.0, '1': 0.000628, '2': 0.00050745, '3': 0.00012328, '4': 0.00571089, '5': 9.656e-05, '6': 6.236e-05, '7': 0.00058693, '8': 0.00022419, '9': 0.00206034}

current_supply_rates: 1.09238065
min_apy: 0.01854118, max_apy: 0.44430607, apy_range: 0.42576489
pool ratio: 0.02130211
to_allocate init: 0.01
to_allocate: 0.01, delta: 0.0
to_allocate: 0.00936218, delta: 0.00063782
to_allocate: 0.00884701, delta: 0.000

In [198]:
import numpy as np
from sturdy.utils.misc import borrow_rate, supply_rate


init_pool = init_assets_and_pools["pools"]["0"]

print(f"Base supply rate: {supply_rate(init_pool['borrow_amount'], init_pool)}")
for pool in simulator.pool_history:
    uid = '0'
    util_rate = pool[uid]["borrow_amount"] / pool[uid]["reserve_size"]
    print(f'util_rate: {util_rate}')
    print(f'supply rate: {supply_rate(util_rate, init_pool)}')


# simulator.pool_history
# print(init_pool)
# for allocation in np.arange(0, 1.1, 0.1):
#     print(f"allocation: {allocation}, {borrow_rate(allocation/init_pool['reserve_size'], init_pool)}")

# pool_data = simulator.pool_history[1]['0']
# util_rate = pool_data["borrow_amount"] / pool_data["reserve_size"]
# pool_yield = 0.95 * supply_rate(
#                 util_rate, init_pool
#             )
# print(pool_yield)

init_assets_and_pools

Base supply rate: 0.23824285714285717
util_rate: 0.6121003506695705
supply rate: 0.07578455048338772
util_rate: 0.5811523988181343
supply rate: 0.06949023725739366
util_rate: 0.5516727046726747
supply rate: 0.06373845711644069
util_rate: 0.5409579195572851
supply rate: 0.06170686585171511
util_rate: 0.5087094678797188
supply rate: 0.05578206136323726
util_rate: 0.5082475519898132
supply rate: 0.0556992653030137
util_rate: 0.5062918583697099
supply rate: 0.055349364612910616
util_rate: 0.5080643945966852
supply rate: 0.05566645145473473
util_rate: 0.5077268294193326
supply rate: 0.055605998507214094
util_rate: 0.5073894468406939
supply rate: 0.05554560943995417
util_rate: 0.5059384321645062
supply rate: 0.055286243510512555
util_rate: 0.48228492656947297
supply rate: 0.05113953365547869
util_rate: 0.5098124476374711
supply rate: 0.0559800011780892
util_rate: 0.5209236841600823
supply rate: 0.05799259681253139
util_rate: 0.5195132185790317
supply rate: 0.05773524351358077
util_rate: 0.51

{'total_assets': 1.0,
 'pools': {'0': {'pool_id': '0',
   'base_rate': 0.04,
   'base_slope': 0.089,
   'kink_slope': 0.19,
   'optimal_util_rate': 0.65,
   'borrow_amount': 0.9,
   'reserve_size': 1.0},
  '1': {'pool_id': '1',
   'base_rate': 0.05,
   'base_slope': 0.034,
   'kink_slope': 0.318,
   'optimal_util_rate': 0.65,
   'borrow_amount': 0.6,
   'reserve_size': 1.0},
  '2': {'pool_id': '2',
   'base_rate': 0.05,
   'base_slope': 0.098,
   'kink_slope': 0.572,
   'optimal_util_rate': 0.65,
   'borrow_amount': 0.8,
   'reserve_size': 1.0},
  '3': {'pool_id': '3',
   'base_rate': 0.03,
   'base_slope': 0.037,
   'kink_slope': 0.391,
   'optimal_util_rate': 0.65,
   'borrow_amount': 0.85,
   'reserve_size': 1.0},
  '4': {'pool_id': '4',
   'base_rate': 0.02,
   'base_slope': 0.063,
   'kink_slope': 0.681,
   'optimal_util_rate': 0.7,
   'borrow_amount': 0.75,
   'reserve_size': 1.0},
  '5': {'pool_id': '5',
   'base_rate': 0.03,
   'base_slope': 0.022,
   'kink_slope': 0.89,
   'op